# without expanding to USA (NC only)

In [2]:
from pyinaturalist import get_observations, get_taxa
import time

NC_PLACE_ID = 30
USA_PLACE_ID = 1

EXPAND_TO_WORLD = [
]

EXPAND_TO_USA = [
]

# Structure: Class -> Sub-Group (Optional) -> Safety Category -> Species Name: ID
dataset_structure = {
    "Spiders (Araneae)": {
        "Theridiidae (Cobweb Spiders)": {
            "MEDICALLY RELEVANT": {
                "Latrodectus mactans (Southern Black Widow)": 47381, 
                "Latrodectus variolus (Northern Black Widow)": 146706
            },
            "MEDICALLY INSIGNIFICANT": {
                "Parasteatoda tepidariorum (Common House Spider)": 120583,
                "Steatoda grossa (False Widow)": 61997,
                "Steatoda triangulosa": 81579,
                "Enoplognatha spp.": 153772
            }
        },
        "Sicariidae (Recluse Spiders)": {
            "MEDICALLY RELEVANT": {
                "Loxosceles reclusa (Brown Recluse)": 50181
            },
            "MEDICALLY INSIGNIFICANT": {
                "Kukulcania spp. (Southern House Spider)": 49330
            }
        }
    },
    "Ticks (Acari)": {
        "Ixodidae (Hard Ticks)": {
            "MEDICALLY RELEVANT": {
                "Amblyomma americanum": 83744,
                "Dermacentor variabilis": 52155,
                "Ixodes scapularis": 60598,
                "Amblyomma maculatum": 180017,
                "Rhipicephalus sanguineus": 229905
            }
        }
    },
    "Mosquitoes (Culicidae)": {
        "MEDICALLY RELEVANT": {
            "Aedes spp.": 62989,
            "Anopheles spp.": 146949,
            "Culex spp.": 130033,
            "Psorophora spp.": 132757, 
            "Coquillettidia spp.": 155399 
        }
    },
    "Stinging Hymenoptera": {
        "Vespidae (Wasps & Hornets)": {
            "MEDICALLY RELEVANT": {
                "Vespula spp. (Yellowjackets)": 61356, 
                "Dolichovespula spp. (Bald-faced hornets)": 52912,
                "Polistes spp. (Paper Wasps)": 52756,
                "Vespa spp. (European Hornet)": 54328
            },
            "MEDICALLY INSIGNIFICANT": {
                "Sceliphron spp. (Mud Daubers)": 121822,
                "Sphecius speciosus (Cicada Killer)": 51954
            }
        }
    },
    "Bees (Apidae)": {
        "MEDICALLY RELEVANT": {
            "Apis mellifera (Honey Bee)": 47219,
            "Bombus spp. (Bumble Bee)": 52775,
            "Xylocopa spp. (Carpenter Bee)": 51111
        }
    },
    "Ants (Formicidae)": {
        "MEDICALLY RELEVANT": {
            "Solenopsis spp.": 67597
        },
        "MEDICALLY INSIGNIFICANT": {
            "Camponotus spp.": 62781,
            "Formica spp.": 47339,
            "Lasius spp.": 69086
        }
    },
    "True Bugs": {
        "Reduviidae (Assassin Bugs)": {
            "MEDICALLY RELEVANT": {
                "Triatoma sanguisuga (Kissing Bug)": 322394
            },
            "MEDICALLY INSIGNIFICANT": {
                "Zelus spp.": 129424,
                "Apiomerus spp.": 61491
            }
        },
        "Coreidae (Leaf-footed Bugs)": {
            "MEDICALLY INSIGNIFICANT": {
                "Coreidae spp.": 51777
            }
        }
    },
    "Stinging Caterpillars": {  
        "MEDICALLY RELEVANT": {
            "Megalopyge opercularis (Puss Caterpillar)": 84185, 
            "Acharia stimulea (Saddleback Caterpillar)": 122228
        },
        "MEDICALLY INSIGNIFICANT": {
            "Saturniidae (Silk Moth Larvae)": 47918,
            "Noctuidae (Typical Moth Larvae)": 48670
        }
    },
    "Blister Beetles (Meloidae)": {
        "MEDICALLY RELEVANT": {
            "Epicauta spp.": 59506
        },
        "MEDICALLY INSIGNIFICANT": {
            "Tenebrionidae (Darkling Beetles)": 53816
        }
    },
    "Scorpions": {
        "MEDICALLY RELEVANT": {
            "Vaejovis carolinianus": 60912
        }
    },
    "Biting Flies": {
        "Tabanidae (Horse & Deer Flies)": {
            "MEDICALLY RELEVANT": {
                "Tabanus spp.": 49704,
                "Chrysops spp.": 49703
            },
            "MEDICALLY INSIGNIFICANT": {
                "Tabanus imitators": 0 # could not find
            }
        },
        "Syrphidae (Hover Flies)": {
            "MEDICALLY INSIGNIFICANT": {
                "Syrphidae spp.": 49995
            }
        }
    },
    "Centipedes (Chilopoda)": {
        "MEDICALLY RELEVANT": {
            "Scolopocryptops spp.": 123665,
            "Lithobius spp.": 122237
        },
        "MEDICALLY INSIGNIFICANT": {
            "Scutigeromorpha (House Centipedes)": 49553
        }
    }
}

# RECURSIVE FUNCTION TO WALK THE TREE
def process_node(name, data, indent_level=0, parent_class=None):
    indent = " " * (indent_level * 4)
    
    # Base Case: We hit a Species ID (Integer or None)
    if isinstance(data, (int, type(None))):
        taxon_id = data
        
        # If placeholder is 0 or None, skip query
        if not taxon_id:
            print(f"{indent}├── {name:<40} [ID: PENDING]")
            return

        params = {
            'taxon_id': taxon_id,
            'quality_grade': "research",
            'count_only': True
        }

        if taxon_id in EXPAND_TO_WORLD:
            # DO NOT set 'place_id' in params to search globally
            loc_str = "WORLD"
        elif taxon_id in EXPAND_TO_USA:
            params['place_id'] = USA_PLACE_ID
            loc_str = "USA  "
        else:
            params['place_id'] = NC_PLACE_ID
            loc_str = "NC   "

        # Caterpillar Filter Logic (Inherited from parent_class context)
        if parent_class and "Caterpillars" in parent_class:
            params['term_id'] = 1 
            params['term_value_id'] = 6
            name += " (Larva Only)"

        try:
            # Execute Query
            response = get_observations(**params)
            count = response['total_results']
            print(f"{indent}├── {name:<40} [ID: {taxon_id}] [{loc_str}]: {count}")
            time.sleep(0.3)
        except Exception as e:
            print(f"{indent}├── {name:<40} [ERROR]: {e}")
        
        return

    # Recursive Case: It's a Folder (Dictionary)
    if indent_level == 0:
        print(f"\n[{name.upper()}]")
        parent_class = name # Set context for children
    else:
        print(f"{indent} {name}")

    for sub_key, sub_value in data.items():
        process_node(sub_key, sub_value, indent_level + 1, parent_class)

if __name__ == "__main__":
    
    for class_name, content in dataset_structure.items():
        process_node(class_name, content)


[SPIDERS (ARANEAE)]
     Theridiidae (Cobweb Spiders)
         MEDICALLY RELEVANT
            ├── Latrodectus mactans (Southern Black Widow) [ID: 47381] [NC   ]: 642
            ├── Latrodectus variolus (Northern Black Widow) [ID: 146706] [NC   ]: 270
         MEDICALLY INSIGNIFICANT
            ├── Parasteatoda tepidariorum (Common House Spider) [ID: 120583] [NC   ]: 235
            ├── Steatoda grossa (False Widow)            [ID: 61997] [NC   ]: 294
            ├── Steatoda triangulosa                     [ID: 81579] [NC   ]: 315
            ├── Enoplognatha spp.                        [ID: 153772] [NC   ]: 5
     Sicariidae (Recluse Spiders)
         MEDICALLY RELEVANT
            ├── Loxosceles reclusa (Brown Recluse)       [ID: 50181] [NC   ]: 29
         MEDICALLY INSIGNIFICANT
            ├── Kukulcania spp. (Southern House Spider)  [ID: 49330] [NC   ]: 1041

[TICKS (ACARI)]
     Ixodidae (Hard Ticks)
         MEDICALLY RELEVANT
            ├── Amblyomma americanum            

# Expanded to USA on scare sets.

In [4]:
from pyinaturalist import get_observations, get_taxa
import time

NC_PLACE_ID = 30
USA_PLACE_ID = 1

EXPAND_TO_WORLD = [
]

EXPAND_TO_USA = [
    # Spiders
    47381, 146706, 120583, 61997, 81579, 153772, 50181,
    # Ticks
    83744, 52155, 60598, 180017, 229905,
    # Mosquitoes
    62989, 146949, 130033, 132757, 155399,
    # Hymenoptera/Ants
    121822, 47339, 69086,
    # Bugs & Beetles
    322394, 61491, 59506, 53816,
    # Caterpillars
    84185, 122228,
    # Scorpions/Flies/Centipedes
    60912, 49704, 49703, 123665, 122237
]

if __name__ == "__main__":
    
    for class_name, content in dataset_structure.items():
        process_node(class_name, content)


[SPIDERS (ARANEAE)]
     Theridiidae (Cobweb Spiders)
         MEDICALLY RELEVANT
            ├── Latrodectus mactans (Southern Black Widow) [ID: 47381] [USA  ]: 7590
            ├── Latrodectus variolus (Northern Black Widow) [ID: 146706] [USA  ]: 3055
         MEDICALLY INSIGNIFICANT
            ├── Parasteatoda tepidariorum (Common House Spider) [ID: 120583] [USA  ]: 3674
            ├── Steatoda grossa (False Widow)            [ID: 61997] [USA  ]: 6755
            ├── Steatoda triangulosa                     [ID: 81579] [USA  ]: 13925
            ├── Enoplognatha spp.                        [ID: 153772] [USA  ]: 3468
     Sicariidae (Recluse Spiders)
         MEDICALLY RELEVANT
            ├── Loxosceles reclusa (Brown Recluse)       [ID: 50181] [USA  ]: 6687
         MEDICALLY INSIGNIFICANT
            ├── Kukulcania spp. (Southern House Spider)  [ID: 49330] [NC   ]: 1041

[TICKS (ACARI)]
     Ixodidae (Hard Ticks)
         MEDICALLY RELEVANT
            ├── Amblyomma americanum 

In [6]:
from pyinaturalist import get_observations, get_taxa
import time

NC_PLACE_ID = 30
USA_PLACE_ID = 1

EXPAND_TO_WORLD = [
    229905, 155399, 122237
]

EXPAND_TO_USA = [
    # Spiders
    47381, 146706, 120583, 61997, 81579, 153772, 50181,
    # Ticks
    83744, 52155, 60598, 180017,
    # Mosquitoes
    62989, 146949, 130033, 132757,
    # Hymenoptera/Ants
    121822, 47339, 69086,
    # Bugs & Beetles
    322394, 61491, 59506, 53816,
    # Caterpillars
    84185, 122228,
    # Scorpions/Flies/Centipedes
    60912, 49704, 49703, 123665
]

if __name__ == "__main__":
    
    for class_name, content in dataset_structure.items():
        process_node(class_name, content)


[SPIDERS (ARANEAE)]
     Theridiidae (Cobweb Spiders)
         MEDICALLY RELEVANT
            ├── Latrodectus mactans (Southern Black Widow) [ID: 47381] [USA  ]: 7590
            ├── Latrodectus variolus (Northern Black Widow) [ID: 146706] [USA  ]: 3055
         MEDICALLY INSIGNIFICANT
            ├── Parasteatoda tepidariorum (Common House Spider) [ID: 120583] [USA  ]: 3674
            ├── Steatoda grossa (False Widow)            [ID: 61997] [USA  ]: 6755
            ├── Steatoda triangulosa                     [ID: 81579] [USA  ]: 13925
            ├── Enoplognatha spp.                        [ID: 153772] [USA  ]: 3468
     Sicariidae (Recluse Spiders)
         MEDICALLY RELEVANT
            ├── Loxosceles reclusa (Brown Recluse)       [ID: 50181] [USA  ]: 6687
         MEDICALLY INSIGNIFICANT
            ├── Kukulcania spp. (Southern House Spider)  [ID: 49330] [NC   ]: 1041

[TICKS (ACARI)]
     Ixodidae (Hard Ticks)
         MEDICALLY RELEVANT
            ├── Amblyomma americanum 